# En el presente script se genera el reporte de asociados para ser cruzado con el de consumos.

In [1]:
# Librerías usadas:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
inicio = datetime.now()
inicio

datetime.datetime(2022, 5, 3, 9, 57, 41, 70111)

In [3]:
# Parámetros de seteo.

# Ruta a la que tiene que salir la información
ruta_base = 'C:\\Users\\lsaucedo\\Desktop\\Brinkmann v2\\'

# Archivo de salida
# archivo_salida = ruta_base + "Stock_Brinkmann_marcado" + ".csv"
archivo_salida = ruta_base + "Stock_Total_marcado" + ".csv"

# Ruta en que se encuentran los archivos:
ruta = ruta_base + 'Stock_SanCor\\'

# Carpeta en que se encuentran las auxiliares:
ruta_aux = 'C:\\modelos\\Auxiliares\\'

# Diagnosticos:
ruta_diag =  "./" + "Diagnosticos.csv"

# Aquí debería cargar las diferentes rutas y filtros de archivos.

# 1- Levantar el excel de stock.

In [4]:
# Levanto el stock existente en una carpeta:

contenido = os.listdir(ruta)

print("Inicio del proceso: " + datetime.now().strftime('%H:%M:%S'))

# Dataframe con el stock:
dfs = pd.DataFrame()

archivos = []
for fichero in contenido:
    if os.path.isfile(os.path.join(ruta, fichero)) and fichero.endswith('.xlsx'):
        df_temp = pd.read_excel(ruta+fichero)
        dfs = pd.concat([dfs, df_temp], axis=0)
        print("Se cargó el archivo: ", fichero, "a las: " + datetime.now().strftime('%H:%M:%S'))

df_temp = []
print("Fin de la carga de archivos")

Inicio del proceso: 09:57:44
Se cargó el archivo:  20.1 Stock 202010.xlsx a las: 10:00:52
Se cargó el archivo:  20.1 Stock 202011.xlsx a las: 10:04:00
Se cargó el archivo:  20.1 Stock 202012.xlsx a las: 10:07:17
Se cargó el archivo:  20.1 Stock 202101.xlsx a las: 10:10:01
Se cargó el archivo:  20.1 Stock 202102.xlsx a las: 10:12:46
Se cargó el archivo:  20.1 Stock 202103.xlsx a las: 10:15:33
Se cargó el archivo:  20.1 Stock 202104.xlsx a las: 10:18:25
Se cargó el archivo:  20.1 Stock 202105.xlsx a las: 10:21:23
Se cargó el archivo:  20.1 Stock 202106.xlsx a las: 10:24:20
Se cargó el archivo:  20.1 Stock 202107.xlsx a las: 10:27:18
Se cargó el archivo:  20.1 Stock 202108.xlsx a las: 10:30:21
Se cargó el archivo:  20.1 Stock 202109.xlsx a las: 10:33:26
Fin de la carga de archivos


## 1.2 Renombrar las columnas

In [9]:
# Se pueden renombrar incluso columnas que no existen (mejorar la eficiencia en función del warning)
dfs.rename( columns = {    'Periodo ID': 'Periodo',
                           'Persona ID': 'Persona',
                           'Persona Fecha Nacimiento': 'Fecha_nac',
                           'Discapacitado ID': 'Discapacidad',
                           'Marca Persona con Diabetes DESC': 'Diabetes',
                           'Marca Persona con Hepatitis DESC': 'Hepatitis',
                           'Marca Persona con PMI DESC': 'PMI',
                           'Marca Persona con Tratamiento Cronico DESC': 'Cronico',
                           'Marca Persona con Tratamiento Oncológico DESC':  'Oncologico',
                           'Sexo ID': 'Sexo',
                           'Plan Sin Segmento ID': 'Plan',
                           'Posicion Asociado DESC': 'Posicion Asociado',
                           'Zona de Promocion DESC': 'Zona de Promocion',
                           'Segmento Agrupado DESC': 'Segmento',
                           'Plan Agrupado Sin Segmento DESC': 'Plan Agrupado',
                           'Stock Asociados Salud': 'Stock',
                           'Edad Periodo ID': 'Edad',
                           'ACE Stock Asociados Salud': 'ACE',
                           'Consumos Costo con Subsidios SSS': 'Consumo3',
                           'Recaudacion Costo con Otros Subsidios': 'Recaudacion3'
                           }, inplace=True)

In [10]:
dfs.Periodo.unique()

array([202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105,
       202106, 202107, 202108, 202109], dtype=int64)

In [11]:
len(dfs)

7189316

In [12]:
dfs.Stock.sum()

7189639

# 2- Agrego archivos auxiliares

In [13]:
# Guardo cantidad de registros iniciales para comparar al final:
len_dfs = len(dfs)

In [14]:
# Agrego la info al df de stock: RANGO EDAD

# Levanto la auxiliar:
Aux_RangoEdad = pd.read_excel(ruta_aux + "Aux_RangoEdad.xlsx")
# Pego la informacion al df stock:
dfs = pd.merge(left = dfs, right = Aux_RangoEdad[["Edad","Rango_Edad"]], left_on='Edad', right_on='Edad', how = "left")
# Elimino la columna adicional -- no hay
# Calculo la diferencia de registros:
len_dfs_edad = len(dfs)
len_dfs_edad - len_dfs
# Elimino Aux_RangoEdad de memoria
Aux_RangoEdad = []

In [15]:
# No debería haber nan
dfs["Rango_Edad"].unique()

array(['Entre 60 y 69', 'Entre 36 y 40', 'Entre 31 y 35', 'Entre 41 y 45',
       'Entre 50 y 59', 'Entre 46 y 49', 'Entre 26 y 30', 'Entre 21 y 25',
       'Entre 16 y 20', 'Entre 11 y 15', 'Entre 6 y 10', 'Entre 0 y 1',
       'Entre 2 y 5'], dtype=object)

In [16]:
# Agrego la info al df de stock: CARTILLA BASE

# Levanto la auxiliar:
Aux_CartillaBase = pd.read_excel(ruta_aux+"Aux_CartillaBase.xlsx")
# Pego la informacion al df stock:
dfs = pd.merge(left = dfs, right = Aux_CartillaBase[["Plan","Modelo_Cartilla"]], left_on='Plan', right_on='Plan', how = "left")
# Elimino la columna adicional -- no hay
# Calculo la diferencia de registros:
len_dfs_cartilla = len(dfs)
len_dfs_cartilla - len_dfs
# Elimino Aux_CartillaBase de memoria
Aux_CartillaBase = []

In [17]:
# No debería haber nan
dfs["Modelo_Cartilla"].unique()

array(['3000', '2000', '500', '4000', '2000E', '1000', '1500', 'AMPS7',
       'AMPS4', '5000', 'C', 'AMPS2', 'AMPS5', 'AMPS3', '4500', 'STAFM',
       'AMPS6', 'OS', 'E2000'], dtype=object)

In [18]:
# Agrego la info al df de stock: COBERTURA MED

# Levanto la auxiliar:
Aux_CobertMedicamentosPlan = pd.read_excel(ruta_aux+"Aux_CobertMedicamentosPlan.xlsx")
# Pego la informacion al df stock:
dfs = pd.merge(left = dfs, right = Aux_CobertMedicamentosPlan[["Plan","Cobertura_med_id"]], left_on='Plan', right_on='Plan', how = "left")
# Elimino la columna adicional -- no hay
# Calculo la diferencia de registros:
len_dfs_medicam = len(dfs)
len_dfs_medicam - len_dfs
# Elimino Aux_CartillaBase de memoria
Aux_CobertMedicamentosPlan = []

In [19]:
# No debería haber nan
dfs["Cobertura_med_id"].unique()

array([306, 302, 315, 301, 319, 303, 304, 308, 310, 305, 307, 312, 313],
      dtype=int64)

In [20]:
# Agrego la info al df de stock: LISTA COPAGOS

# Levanto la auxiliar:
Aux_ListaCopagos = pd.read_excel(ruta_aux+"Aux_ListaCopagos.xlsx")
# Pego la informacion al df stock:
dfs = pd.merge(left = dfs, right = Aux_ListaCopagos[["Plan","Lista_PrecioDesc"]], left_on='Plan', right_on='Plan', how = "left")
# Elimino la columna adicional -- no hay
# Calculo la diferencia de registros:
len_dfs_copagos = len(dfs)
len_dfs_copagos - len_dfs
# Elimino Aux_CartillaBase de memoria
Aux_ListaCopagos = []

# 3- Agregar Diagnósticos

Existe un script que realiza el cálculo de patologías y GC. Aquí solamente lo cargamos y cruzamos.

In [21]:
ruta_diag

'./Diagnosticos.csv'

In [22]:
# Levanto la auxiliar:
dfdiag = pd.read_csv(ruta_diag)

# Pego la informacion al df stock:
dfs = pd.merge(left = dfs, right = dfdiag[["Persona",'GC', 'Rango_Cons', 
              'algun_diag', 'prop_diag', 'Marca_Diagnostico']], left_on='Persona', right_on='Persona', how = "left")
# Elimino la columna adicional -- no hay
# Calculo la diferencia de registros:
len_dfs_diag = len(dfs)
len_dfs_diag - len_dfs
# Elimino Aux_CartillaBase de memoria
dfdiag = []
len_dfs_diag - len_dfs

0

Transformamos los diag. conocidos:

In [23]:
# Reemplazo valores de filas con replace:
dfs['Diabetes'] = dfs['Diabetes'].replace(['No Diabético', 'Diabético'], [0, 1])
dfs['Hepatitis'] = dfs['Hepatitis'].replace(['Sin Hepatitis', 'Con Hepatitis'], [0, 1])
dfs['PMI'] = dfs['PMI'].replace(['NO', 'SI'], [0, 1])
dfs['Cronico'] = dfs['Cronico'].replace(['NO', 'SI'], [0, 1])
dfs['Oncologico'] = dfs['Oncologico'].replace(['Sin Marca', 'Paciente Oncológico'], [0, 1])
dfs['algun_diag'] = dfs['Marca_Diagnostico'].replace(['Sin Diagnostico', 'Con Diagnostico'], [0, 1])


In [24]:
dfs.columns

Index(['Periodo', 'Persona', 'Fecha_nac', 'Parentesco IdParentesco', 'Edad',
       'Sexo', 'Discapacidad', 'Posicion Asociado ID', 'Zona de Promocion ID',
       'Segmento Agrupado ID', 'Segmento', 'Plan', 'Plan Agrupado',
       'Tipo de Venta DESC', 'Tipo de Venta ID', 'Subtipo de Venta DESC',
       'Subtipo de Venta ID', 'Diabetes', 'Hepatitis', 'PMI', 'Cronico',
       'Oncologico', 'Stock', 'ACE', 'Rango_Edad', 'Modelo_Cartilla',
       'Cobertura_med_id', 'Lista_PrecioDesc', 'GC', 'Rango_Cons',
       'algun_diag', 'prop_diag', 'Marca_Diagnostico'],
      dtype='object')

In [56]:
# en caso que se quiera exportar sin la td previa.
# dfs.to_csv("Stock_brinkmann.xlsx", encoding = 'latin1', decimal=',')

# 4- Exportar la información en ".csv"

## Exportamos parcialmente el stock de brinkmann
Hasta aquí se puede hacer para todo el stock.

In [25]:
# Reemplazo los datos faltantes:
dfs["GC"].fillna(0, inplace = True)
dfs["Rango_Cons"].fillna("Muy Bajo", inplace = True)
dfs["algun_diag"].fillna(0, inplace = True)
dfs["prop_diag"].fillna(0, inplace = True)
dfs["Marca_Diagnostico"].fillna("Sin Diagnostico", inplace = True)

# Reemplazo los valores mayores a 1 por 1 para que quede marca de gc 1 o 0
#en lugar de cantidad de veces siendo GC..
# Es un reemplazar que funciona al revez: Reemplaza todos los valores que NO CUMPLEN la condición
dfs['GC'] = dfs['GC'].where(dfs['GC'] == 0 , 1)

In [26]:
dfs['Parentesco IdParentesco'].unique()

array([ 0,  1, 90,  5,  9,  8,  2,  3, 99, 92, 98,  4, 91,  6,  7, 10, 12],
      dtype=int64)

In [27]:
# Agregamos los parentescos de los hijos:
# Método np.select()

condiciones = [
    (dfs['Parentesco IdParentesco'] == 3),
    (dfs['Parentesco IdParentesco'] == 4),
    (dfs['Parentesco IdParentesco'] == 5),
    (dfs['Parentesco IdParentesco'] == 6),
    (dfs['Parentesco IdParentesco'] == 7)]

# Lista de resultados en función de las selecciones
valores_condic = [1,1,1,1,1]     

# Generación de la columna calculada
dfs['Marca Hijo'] = np.select(condiciones, valores_condic, default=0) # se analizaron y surgen de consumo 3 == 0

In [28]:
dfs.Stock.sum()

7189639

In [29]:
len(dfs)

7189316

In [59]:
##############################################
##############################################
# SI QUEREMOS EXPORTAR POR PERSONA - PERIODO

# Exportación de archivo sin hacer td:
dfs.to_csv(archivo_salida, encoding = 'latin1', decimal=',', index = False)

In [ ]:
##############################################
##############################################
# SI QUEREMOS EXPORTAR POR APERTURAS:

In [56]:
# Defino los campos de segmentación:

# QUEDA PENDIENTE SEGMENTO EN LA APERTURA
segmentacion = ([ 'Periodo', 'Posicion Asociado ID', 'Plan Agrupado',
                'Sexo', 'Rango_Edad','Rango_Cons'])


# 'Cobertura_med_id', 
#   'Lista_PrecioDesc',
# 'Modelo_Cartilla',
# 'Tipo de Venta ID',
# , 'Segmento'
# , 'Zona de Promocion ID'
# , 'Marca_Diagnostico'


# dejo para probar procesamientos más rápido (debugging)
segmentacion2 = ['Periodo', 'Sexo']


# Segementación inicial SIN SEGMENTO:
segmentacion3 = (['Periodo', 'Posicion Asociado ID', 'Zona de Promocion ID', 'Plan Agrupado',
                'Sexo', 'Rango_Edad', 'Tipo de Venta ID','Cobertura_med_id', 'Modelo_Cartilla', 'Lista_PrecioDesc',
                'Rango_Cons', 'Marca_Diagnostico'])


# Defino las métricas a calcular
metricas = ({   'Stock':np.sum,
                'Edad':np.sum,
                'ACE':np.sum,
                'Oncologico':np.sum,
                'Cronico':np.sum,
                'PMI':np.sum,
                'Hepatitis':np.sum,
                'Diabetes':np.sum,
                'GC':np.sum,
                'algun_diag':np.sum,
                'Marca Hijo':np.sum,
            })


metricas2 = ({   'Stock':[np.sum,len],
                'Edad':np.sum,
                'ACE':np.sum,
                          })


# Genero la tabla de salida
td = pd.pivot_table(dfs, index = segmentacion, 
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas)


# Genero un dataframe con la td
base_asociados = pd.DataFrame(td)


In [58]:
# Exporto la información en csv:
base_asociados.to_csv(archivo_salida , encoding = 'latin1', decimal=',', index = False)

# 5- Adicional: Pegar los consumos para exportar

In [64]:
# leemos el df consumo procesado
dfcp = pd.read_csv('./PerPrest-Persona-Consumo-Bnknn_v2.csv', encoding = 'latin1', decimal=',')
dfcp.head()

,Periodo_Prest,Persona,Cantidad Total,Consumo Total,Prest. Dist. Total,Cantidad Amb.,Cantidad Int.,Cantidad Medic.,Cantidad NC.,Cantidad NA,...,Prest. Farmacia,Prest. No Considerar.1,Prest. Otras vias,Prest. Provision,Cantidad NO PMO,Cantidad PMO,Consumo NO PMO,Consumo PMO,Prest. NO PMO,Prest. PMO
0,202010,266,8,1101.97,6,2,0,0,2,4,...,0,6,0,0,1,7,1.30,1100.67,1,5
1,202010,452,5,3366.78,5,1,0,0,3,1,...,0,5,0,0,2,3,1.30,3365.48,2,3
2,202010,779,22,4835.15,19,13,1,2,2,4,...,2,17,0,0,3,19,1589.25,3245.90,3,16
3,202010,780,27,4422.58,8,21,0,0,2,4,...,0,8,0,0,11,16,3151.30,1271.28,2,6
4,202010,1219,11,5360.73,10,4,0,0,3,4,...,0,10,0,0,2,9,1.30,5359.43,2,8


In [65]:
dff = pd.merge(left = dfs, right = dfcp, 
               left_on=("Persona","Periodo"), right_on=("Persona","Periodo_Prest"), how = "left")

In [66]:
dff["Consumo Total"].sum()

102792475.71000001

In [67]:
dfcp["Consumo Total"].sum()

102763070.06

In [68]:
dfcp.Periodo_Prest.unique()

array([202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105,
       202106, 202107, 202108, 202109], dtype=int64)

In [69]:
dff = dff.fillna(0)

In [70]:
# Exportamos el df final
dff = dff.to_csv("Reporte_Brinkmann.csv", encoding = 'latin1', decimal=',', index = False)

# Borrar todas las variables en memoria

In [ ]:
# https://www.delftstack.com/es/howto/python/how-to-clear-variables-in-ipython/#:~:text=Podemos%20usar%20%25reset%20para%20borrar,para%20borrar%20una%20variable%20espec%C3%ADfica.
%reset